In [ ]:
import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import dill, os
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
%matplotlib inline
import matplotlib
from mpl_toolkits.mplot3d import axes3d
import seaborn as sns
matplotlib.rcParams['savefig.dpi'] = 2 * matplotlib.rcParams['savefig.dpi']
from bokeh.plotting import figure, show, output_notebook, output_file

In [ ]:
RawCount = dill.load(open("../StatsData/RawCount.pickle"))

###############################
# Data transformation
###############################
UserScore = RawCount
for col in UserScore.columns:
    # Method 1: quantile normalization
    #UserScore[col] = [(stats.percentileofscore(RawCount[col], a, 'strict')/100) for a in RawCount[col]]
    
    # Method 2: exponential distribution
    a = 1-np.exp(-RawCount[col]/np.mean(RawCount[col]))
    a[a<0]=0 # set negative scores to 0
    UserScore[col] = a
#UserScore
len(RawCount)

In [ ]:
# Further transformation??? 
UserScorePro = UserScore[UserScore.max(axis=1) >= 0]
#UserScoreAll = UserScorePro.copy()
#UserScoreAll['max'] = UserScorePro.max(axis=1)
#UserScoreAll.sort([ 'max'])

In [ ]:
##################################
# PCA -> K means clustering
##################################

X = UserScorePro
k=6
n_pca = 100
X = PCA(n_components=n_pca).fit_transform(X)
clusters = KMeans(n_clusters=k)
clusters.fit(X) # all_schools is a n * m matrix, where n is the number of samples, m is the number of 
centroids = clusters.cluster_centers_
labels = clusters.labels_

In [ ]:
newUserScore = UserScore.reset_index(drop=True)
ff = {}
group = pd.DataFrame(np.arange(k), columns=['group ID'])
Usercount = []
groupinfo = {}
for i in range(k):
    gg = newUserScore.loc[np.where(labels==i)]
    ff[i] = pd.merge(pd.DataFrame(gg.mean(),columns=['mean']), pd.DataFrame(gg.median(),columns=['median']), 
                     left_index=True, right_index=True, how='outer')
    Usercount.append(len(newUserScore.loc[np.where(labels==i)]))
    groupinfo[i] = ff[i].sort('mean',ascending=0).head(20)

In [ ]:
colors = ['red','blue','green','yellow', 'orange', 'magenta','black']
for i in range(k):
    print '____________________________________________'
    print 'Group', colors[i], ':', Usercount[i], 'users'
    print  groupinfo[i]
    print '\n'

In [ ]:
   
#TOOLS="resize,crosshair,pan,wheel_zoom,box_zoom,reset,tap,previewsave,box_select,poly_select,lasso_select"
output_notebook()
#p = figure(tools=TOOLS)


#output_file("clustering.html", title="line plot example")
p = figure(title="Stats Overflow User clustering", x_axis_label='1st component', y_axis_label='2nd component')
    
for j in range(k):
    ds = X[np.where(labels==j)]
    p.scatter(ds[:,0], ds[:,1], fill_alpha=0.6, color=colors[j])

for j in range(k):
    p.scatter(centroids[j,0],centroids[j,1], size=12,  marker='x',color='black')


show(p)


In [ ]:
import xml.etree.ElementTree as ET
tree = ET.parse('../StatsData/Users.xml')
root = tree.getroot()
chil = root.getchildren()
UsersDF = pd.DataFrame([ch.attrib for ch in chil])

In [ ]:
groupUsers = UserScore.reset_index().loc[np.where(labels==1)]
groupUsers['sum'] = groupUsers.sum(axis=1)
ExpertIDs = groupUsers.sort('sum',ascending=0)['OwnerUserId'][0:10].values
print ExpertIDs

In [ ]:
for ids in ExpertIDs:
    user = UsersDF[UsersDF['Id'] ==ids]
    name = user['DisplayName'].values[0]
    web = user['WebsiteUrl'].values[0]
    reputation = user['Reputation'].values[0]
    print 'Display Name:', name,'(Reputation:',reputation,')'
    print 'Website url:', web


In [ ]:
groupUsers = UserScore.reset_index().loc[np.where(labels==4)]
groupUsers['sum'] = groupUsers.sum(axis=1)
ExpertIDs = groupUsers.sort('sum',ascending=0)['OwnerUserId'][0:10].values
print ExpertIDs
for ids in ExpertIDs:
    user = UsersDF[UsersDF['Id'] ==ids]
    name = user['DisplayName'].values[0]
    web = user['WebsiteUrl'].values[0]
    reputation = user['Reputation'].values[0]
    print 'Display Name:', name,'(Reputation:',reputation,'),',
    print 'URL:', web